In [1]:
import nptdms as tdms
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import time

# transfer TDMS to CSV

In [ ]:
# voltage 
dir_file = "./电流电压/" 
filename = "Voltage-Current-2019-11-19T08-55-08"
suffix = ".tdms"
dir_save = "./voltage_current_csv/"
group_name = 'Voltage-Current'


for (_,_,files) in os.walk(dir_file):
    df_file = pd.DataFrame()
    df_file['file'] = files
    df_file.to_csv("./temp/voltage_files.txt",index = False)
    for file in files:
        if file[-5:]==".tdms":
            filename = file[:-5]
            file_read = dir_file+filename+suffix
            file_save = dir_save+filename+".csv"
            try:
                # read file
                tdms_file = tdms.TdmsFile(file_read)
                # transform tdms into dataframe
                df = tdms_file.as_dataframe()
                # change the columns name
                df.columns = ["time","Ic","U_IGBT3_1","U_IGBT3_2","Uce"]

                # drop some nansense columns
    #             df.drop(["N0","N1"],axis = 1,inplace = True)
                try:
                    df.dropna(axis = 0,inplace = True)
                except:
                    pass
                # transform the string time into timestamp
                timestamp = []
                for t in df.time:
                    if t != "":
                        timestamp.append(time.mktime(time.strptime(t[0:-7],"%Y/%m/%dT%H:%M:%S"))+ float(t[-7:]))
                    else:
                        timestamp.append(0)
                df["timestamp" ]= timestamp
                df["timestamp"].replace(0,np.NaN,inplace = True)
                # save file
                df.to_csv(file_save)
            except:
                print(file)
                pass
      

Voltage-Current-2019-12-27T08-04-02.tdms
Voltage-Current-2019-12-27T10-11-58.tdms
Voltage-Current-2019-12-27T11-02-27.tdms
Voltage-Current-2019-12-27T11-52-52.tdms
Voltage-Current-2019-12-27T12-43-08.tdms
Voltage-Current-2019-12-27T13-33-21.tdms
Voltage-Current-2019-12-27T14-23-32.tdms
Voltage-Current-2019-12-27T15-13-48.tdms
Voltage-Current-2020-01-09T11-34-21.tdms
Voltage-Current-2020-01-09T15-14-47.tdms


In [ ]:
# temperature
dir_file = "./温度/" 
filename = "Temperature-2019-11-19T08-55-08"
suffix = ".tdms"
dir_save = "./temperature_csv/"


for (_,_,files) in os.walk(dir_file):
    df_file = pd.DataFrame()
    df_file['file'] = files
    df_file.to_csv("./temp/temperature_files.txt",index = False)
    for file in files:
        if file[-5:]==".tdms":
            filename = file[:-5]
            file_read = dir_file+filename+suffix
            file_save = dir_save+filename+".csv"
            
            try:
                # read file
                tdms_file = tdms.TdmsFile(file_read)
                # transform tdms into dataframe
                df = tdms_file.as_dataframe()
                # change the columns name
                try:
                    df.columns = ["time","Tbase_Cu","Tsink_IGBT3_2","Tsink_IGBT3_1","Tj_IGBT3_2","Tj_IGBT3_1"]
                except:
                    df.columns = ["time","Tbase_Cu","Tsink_IGBT3_2","Tsink_IGBT3_1","Tj_IGBT3_2","Tj_IGBT3_1",
                                  "Tsink_IGBT4_2","Tsink_IGBT4_1","Tj_IGBT4_1"]
                # drop some nansense columns
                try:
                    df.dropna(axis = 0,inplace = True)
                except:
                    pass
                # transform the string time into timestamp
                timestamp = []
                for t in df.time:
                    if t != "":
                        timestamp.append(time.mktime(time.strptime(t[0:-7],"%Y/%m/%dT%H:%M:%S"))+ float(t[-7:]))
                    else:
                        timestamp.append(0)
                df["timestamp" ]= timestamp
                df["timestamp"].replace(0,np.NaN,inplace = True)
                # save file
                df.to_csv(file_save)
                
            except:
                print(file)
                pass

## merge temperature and voltage data on time 

In [3]:
dir_temp = "./temperature_csv/" 
dir_volt = "./voltage_current_csv/" 
prefix_temp = "Temperature-"
prefix_volt = "Voltage-Current-"
time_str = "2019-12-09T17-58-55"
suffix = ".csv"
dir_save = "./temperature_voltage_current_csv/"

file_err = []
for (_,_,files) in os.walk(dir_volt):
    for file in files:
        time_str = file[-23:-4]
        # read file
        try:
            file_temp = dir_temp+prefix_temp+time_str+suffix
            file_volt = dir_volt+prefix_volt+time_str+suffix
            volt_df = pd.read_csv(file_volt,index_col = 0)
            temp_df = pd.read_csv(file_temp,index_col = 0)
        except:
            print("read error: ",time_str)
            file_err.append(file)
            
        try:
            # 去除不规整的volt数据
            select_df = volt_df.dropna(axis = 0)
            l = select_df.index.values
            m= np.where(l[1:]-l[:-1]==51)
            volt_df = volt_df.loc[l[m[0][0]]:,:]
        except:
            print("select error: ",time_str)
            file_err.append(file)
        
        try:
            # 重新定义时间
            volt_df["t"] = volt_df["timestamp"].round(1)
            volt_df.drop(["timestamp","time"],axis = 1,inplace = True )
            temp_df["t"] = temp_df["timestamp"].round(1)
            temp_df.drop(["timestamp","time"],axis = 1,inplace = True )
        except:
            print("def t error: ",time_str)
            file_err.append(file)
        
        try:
            # merge temp to voltage
            merge_df = pd.merge(volt_df,temp_df,on = "t", how = "left")
            # save
            merge_df.to_csv(file_volt,index = False)
            # merge voltage to temperature
            merge_df = pd.merge(temp_df,volt_df,on = "t", how = "left")
            # save
            merge_df.to_csv(file_temp,index = False)
        except:
            print("merge error: ",time_str)
            file_err.append(file)
            
err_file_df = pd.DataFrame()
err_file_df["file"] = file_err
err_file_df.to_csv("./temp/error_files.txt",index = False)

read error:  2019-12-19T21-04-48
def t error:  2019-12-19T21-04-48
read error:  2019-12-31T09-31-35
def t error:  2019-12-31T09-31-35
read error:  2019-12-31T09-42-48
def t error:  2019-12-31T09-42-48


# cut into cycle

In [4]:
dir_volt = "./voltage_current_csv/" 
prefix_volt = "Voltage-Current-"
time_str = "2019-12-09T17-58-55"
suffix = ".csv"

n_round = 0
for (_,_,files) in os.walk(dir_volt):
    for file in files:
        time_str = file[-23:-4]
        try:
            # read file
            file_volt = dir_volt+prefix_volt+time_str+suffix
            volt_df = pd.read_csv(file_volt)
            # get index of max T and min T
            select_df = volt_df.dropna(inplace = False)
            l = select_df["Tj_IGBT3_1"].values
            l[1:-1] = (l[:-2]+l[1:-1]+l[2:])/3
            ix = select_df.index.values  # index of temp
            m = l[1:]-l[:-1]> 0
            m = m.astype(np.int8)
            n = m[1:]-m[:-1]
            n = np.insert(n,0,values = 1)
            n = np.append(n,values = 1)  # 1 0 0 0 -1 0 0 1 0 0 -1 0 0 1 //the same size as ix
            i_Tmax = np.where(n==-1)[0]  # 
            i_Tmin = np.where(n==1)[0]

            # cut and save
            for i in range(len(i_Tmax)):
                n_round += 1
                dur = i_Tmin[i+1]-i_Tmin[i]
                Tmax = l[i_Tmax[i]]
                Tmin1 = l[i_Tmin[i]]
                Tmin2 = l[i_Tmin[i+1]]
            #     print(n_round,dur,Tmax,Tmin1,Tmin2)
                df = volt_df.loc[ix[i_Tmin[i]+1]:ix[i_Tmin[i+1]],:]
                df.to_csv("./voltage_current_cycle/"+str(n_round)+"-"+str(dur)+"-"+str(int(Tmin1))+"-"+str(int(Tmax))+"-"+str(int(Tmin2))+".csv",index = False)
        except:
            print(time_str)



In [5]:
dir_temp = "./temperature_csv/" 
prefix_temp = "Temperature-"
# time_str = "2019-12-09T17-58-55"
suffix = ".csv"

n_round = 0
for (_,_,files) in os.walk(dir_volt):
    for file in files:
        time_str = file[-23:-4]
        try:
            # read file
            file_volt = dir_temp+prefix_temp+time_str+suffix
            volt_df = pd.read_csv(file_volt)
            # get index of max T and min T
#             select_df = volt_df.dropna(inplace = False)
            l = volt_df["Tj_IGBT3_1"].values
            l[1:-1] = (l[:-2]+l[1:-1]+l[2:])/3
            ix = volt_df.index.values
            m = l[1:]-l[:-1]> 0
            m = m.astype(np.int8)
            n = m[1:]-m[:-1]
            n = np.insert(n,0,values = 1)
            n = np.append(n,values = 1)
            i_Tmax = np.where(n==-1)[0]
            i_Tmin = np.where(n==1)[0]

            # cut and save
            for i in range(len(i_Tmax)):
                n_round += 1
                dur = (i_Tmin[i+1]-i_Tmin[i])/10
                Tmax = l[i_Tmax[i]]
                Tmin1 = l[i_Tmin[i]]
                Tmin2 = l[i_Tmin[i+1]]
            #     print(n_round,dur,Tmax,Tmin1,Tmin2)
                df = volt_df.loc[ix[i_Tmin[i]+1]:ix[i_Tmin[i+1]],:]
                df.to_csv("./temperature_cycle/"+str(n_round)+"-"+str(int(dur))+"-"+str(int(Tmin1))+"-"+str(int(Tmax))+"-"+str(int(Tmin2))+".csv",index = False)
        except:
            print(time_str)

